In [8]:
# 세션 윈도우 테스트 초기화
import json
import random
from datetime import datetime, timedelta
from kafka import KafkaProducer

producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

In [2]:
# Redis 연결 및 키 확인
import redis
import json
from pprint import pprint

# Redis 클라이언트 생성
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

try:
    r.ping()
    print('Redis 연결 성공')
except Exception as e:
    print(f"Redis 연결 실패: {e}")


Redis 연결 성공


In [69]:

# 사용자 111: 하나의 긴 세션 (5분간 활동)
print("사용자 111 세션 생성 (단일 세션, 5분간 활동):")

user111_events = [
    {"type": "view", "offset_minutes": 0},
    {"type": "view", "offset_minutes": 1}, 
    {"type": "cart", "offset_minutes": 3},
    {"type": "purchase", "offset_minutes": 5}
]

# 기준 시간 설정
base_time = datetime.now()
print(base_time)

for event_info in user111_events:
    event_time = base_time + timedelta(minutes=event_info["offset_minutes"])
    
    event = {
        "event_time": event_time.isoformat(),
        "event_type": event_info["type"],
        "price": round(random.uniform(10, 500), 2),
        "user_id": "111",
    }
    
    producer.send("ecommerce-events", value=event)
    print(f"  {event_info['type']} - {event_time.strftime('%H:%M:%S')} (+{event_info['offset_minutes']}분)")

producer.flush()
print("✅ 사용자 111 이벤트 전송 완료\n")


사용자 111 세션 생성 (단일 세션, 5분간 활동):
2025-07-04 22:53:46.767808
  view - 22:53:46 (+0분)
  view - 22:54:46 (+1분)
  cart - 22:56:46 (+3분)
  purchase - 22:58:46 (+5분)
✅ 사용자 111 이벤트 전송 완료



In [72]:
event_time = datetime.now() + timedelta(minutes=15)
print(event_time)
event = {
        "event_time": event_time.isoformat(),
        "event_type": event_info["type"],
        "price": round(random.uniform(10, 500), 2),
        "user_id": "111",
    }
print(f"  {event_info['type']} - {event_time.strftime('%H:%M:%S')} 분)")
producer.send("ecommerce-events", value=event)
producer.flush()

2025-07-04 23:10:08.342071
  purchase - 23:10:08 분)


In [78]:
user_keys = r.keys("*")
for key in user_keys[:10]:  # 처음 10개만 표시
    print(f"- {key}")

- session_features:111:1751637308559
- user_features:111


In [76]:
r.hgetall("session_features:111:1751637308559")

{'duration_seconds': '300', 'user_id': '111', 'event_count': '4'}

In [17]:
r.hgetall("session_features:111:1751556831315")

{'duration_seconds': '300', 'user_id': '111', 'event_count': '12'}